# How to add semantic search to your agent's memory

## Load LLM

In [27]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="qwen2.5:14b",
    base_url='http://127.0.0.1:11434/v1',
    api_key='ollama',
    temperature=0)

## Index Store 생성

- Semantic 검색이 가능한 메모리 저장소 생성

In [28]:
from langchain.embeddings import init_embeddings
from langgraph.store.memory import InMemoryStore

embedings = init_embeddings('huggingface:BAAI/bge-m3')
store = InMemoryStore(
    index={
        'embed': embedings,
        'dims' : 1024
    }
)

## Store some memories

In [29]:
store.put(('user_123','memories'),'1',{'text': 'I love pizza'})
store.put(('user_123','memories'),'2',{'text': 'I prefer Italian food'})
store.put(('user_123','memories'),'3',{'text': "I don't like spicy food"})
store.put(('user_123','memories'),'3',{'text': 'I am studying econometrics'})
store.put(('user_123','memories'),'3',{'text': 'I am a plumber'})

## Search memories using natural language

In [30]:
memories = store.search(('user_123', 'memories'), query="I'm hungry", limit=3)

for memory in memories:
    print(f"Memory: {memory.value['text']} (similarity: {memory.score})")

Memory: I prefer Italian food (similarity: 0.6429529122454709)
Memory: I am a plumber (similarity: 0.6412285135022295)
Memory: I love pizza (similarity: 0.6045187378022228)


# Using in your agent

In [32]:
from typing import Optional

from langgraph.store.base import BaseStore

from langgraph.graph import START, MessagesState, StateGraph

llm = model

def chat(state, *, store: BaseStore):
    # Search based on user's last message

    items = store.search(
        ('user_123', 'memories'), query=state['messages'][-1].content, limit=2
    )

    memories = '\n'.join(item.value['text'] for item in items)
    memories = f"## Memories of user\n{memories}" if memories else ""
    response = llm.invoke(
        [
            {'role': 'system' , 'content': f"You are a helpful assistant.\n{memories}"},
            *state['messages'],
        ]
    )
    return {'messages' : [response]}

builder = StateGraph(MessagesState)
builder.add_node(chat)
builder.add_edge(START, 'chat')
graph = builder.compile(store=store)

for message, metadata in graph.stream(
    input = {'messages' : [{'role': 'user' , 'content' : "I'm hungry"}]},
    stream_mode='messages'
):
    print(message.content, end = "")

Since you mentioned that you prefer Italian food, how about trying some pasta or pizza for lunch? If you're looking for something quick and easy, maybe a margherita pizza or spaghetti with marinara sauce would be perfect. Do you have any favorite spots nearby, or should I suggest some places to order from?

# Using in `create_react_agent`

In [17]:
import uuid
from typing import Optional

from langgraph.prebuilt import InjectedStore
from langgraph.store.base import BaseStore
from typing_extensions import Annotated

from langgraph.prebuilt import create_react_agent

def prepare_messages(state, *, store: BaseStore):
    items = store.search(
        ("user_123", "memories"), query=state['messages'][-1].content, limit=2
    )

    memories = "\n".join(item.value['text'] for item in items)
    memories = f"## Memories of user\n{memories}" if memories else ""
    
    return [
        {"role": "system", "content" : f"You are a helpful assistant.\n{memories}"}
    ] + state['messages']

def upsert_memory(
        content: str,
        *,
        memory_id: Optional[uuid.UUID] = None,
        store: Annotated[BaseStore, InjectedStore],
):
    """Upsert a memory in the database"""
    # The LLM can use this tool to store a new memory
    mem_id = memory_id or uuid.uuid4()
    store.put(
        ("user_123", "memories"),
        key = str(mem_id),
        value = {'text': content}
    )
    return f"Stroed memory {mem_id}"

In [18]:
agent = create_react_agent(
    model,
    tools=[upsert_memory],
    prompt=prepare_messages,
    store=store
)

In [19]:
for msg, metadata in agent.stream(
    input={"messages" : [{"role": "user", "content" : "I'm hungry"}]},
    stream_mode="messages",
):
    print(msg.content, end ="")

Since you prefer Italian food, how about we find some Italian restaurants nearby? Would you like me to help with that? If so, I'll need your location to proceed. Alternatively, if you're looking for a quick meal, there might be some Italian dishes that are easy to make at home. Let me know what you'd prefer!

# Advanced Usage

## Multi-vector indexing

In [ ]:
from langchain.embeddings import init_embeddings
from langgraph.store.memory import InMemoryStore

embeddings = init_embeddings('huggingface:BAAI/bge-m3')

# Configure store to embed both memory content and emotional context
store = InMemoryStore(
    index = {'embed' : embeddings , 'dims': 1024, "fields": ["memory", "emotional_context"]}
)

# Store memories with different content/emotion pairs
store.put(
    ("user_123", "memories"),
    "mem1",
    {
        "memory" : "Had pizza with friends at Mario's",
        "emotional_context" : "felt happy and connected",
        "this_isnt_idexed" : "I perfer ravioli though"
    }
)

store.put(
    ("user_123", "memories"),
    "mem2",
    {
        "memory": "Ate alone at home",
        "emotional_context": "felt a bit lonely",
        "this_isnt_indexed": "I like pie",
    },
)

results = store.search(
    ("user_123","memories"), query="times they felt isolated", limit=1
)

print("Expect mem2")
for r in results:
    print(f"Item: {r.key}; Score ({r.score})")
    print(f"Memory: {r.value['memory']}")
    print(f"Emotion: {r.value['emotional_context']}\n")

print("Expect mem1")
results = store.search(("user_123", "memories"), query="fun pizza", limit=1)
for r in results:
    print(f"Item: {r.key}; Score ({r.score})")
    print(f"Memory: {r.value['memory']}")
    print(f"Emotion: {r.value['emotional_context']}\n")

print("Expect random lower score (ravioli not indexed)")
results = store.search(("user_123", "memories"), query="ravioli", limit=1)
for r in results:
    print(f"Item: {r.key}; Score ({r.score})")
    print(f"Memory: {r.value['memory']}")
    print(f"Emotion: {r.value['emotional_context']}\n") ## fields 에 포함시켜준 애만 Indexing이 된다.

Expect mem2
Item: mem2; Score (0.7748096097731049)
Memory: Ate alone at home
Emotion: felt a bit lonely

Expect mem1
Item: mem1; Score (0.6843336968592125)
Memory: Had pizza with friends at Mario's
Emotion: felt happy and connected

Expect random lower score (ravioli not indexed)
Item: mem1; Score (0.536263103505899)
Memory: Had pizza with friends at Mario's
Emotion: felt happy and connected



## Override fields at storage time

In [22]:
store = InMemoryStore(
    index=(
        {
            'embed': embeddings,
            'dims' : 1024,
            'fileds' : ['memory'],
        }
    )
)

# Store one memory with default indexing
store.put(
    ("user_123", "memories"),
    "mem1",
    {"memory": "I love spicy food", "context": "At a Thai restaurant"},
)

# Store another overriding which fields to embed
store.put(
    ("user_123", "memories"),
    "mem2",
    {"memory": "The restaurant was too loud", "context": "Dinner at an Italian place"},
    index=["context"],  # Override: only embed the context
)

# Search about food - matches mem1 (using default field)
print("Expect mem1")
results = store.search(
    ("user_123", "memories"), query="what food do they like", limit=1
)
for r in results:
    print(f"Item: {r.key}; Score ({r.score})")
    print(f"Memory: {r.value['memory']}")
    print(f"Context: {r.value['context']}\n")

# Search about restaurant atmosphere - matches mem2 (using overridden field)
print("Expect mem2")
results = store.search(
    ("user_123", "memories"), query="restaurant environment", limit=1
)
for r in results:
    print(f"Item: {r.key}; Score ({r.score})")
    print(f"Memory: {r.value['memory']}")
    print(f"Context: {r.value['context']}\n")

Expect mem1
Item: mem2; Score (0.6082269820388908)
Memory: The restaurant was too loud
Context: Dinner at an Italian place

Expect mem2
Item: mem2; Score (0.6742498352381826)
Memory: The restaurant was too loud
Context: Dinner at an Italian place



## Disable Indexing for Specific Memories

In [23]:
store = InMemoryStore(index={"embed": embeddings, "dims": 1536, "fields": ["memory"]})

# Store a normal indexed memory
store.put(
    ("user_123", "memories"),
    "mem1",
    {"memory": "I love chocolate ice cream", "type": "preference"},
)

# Store a system memory without indexing
store.put(
    ("user_123", "memories"),
    "mem2",
    {"memory": "User completed onboarding", "type": "system"},
    index=False,  # Disable indexing entirely
)

# Search about food preferences - finds mem1
print("Expect mem1")
results = store.search(("user_123", "memories"), query="what food preferences", limit=1)
for r in results:
    print(f"Item: {r.key}; Score ({r.score})")
    print(f"Memory: {r.value['memory']}")
    print(f"Type: {r.value['type']}\n")

# Search about onboarding - won't find mem2 (not indexed)
print("Expect low score (mem2 not indexed)")
results = store.search(("user_123", "memories"), query="onboarding status", limit=1)
for r in results:
    print(f"Item: {r.key}; Score ({r.score})")
    print(f"Memory: {r.value['memory']}")
    print(f"Type: {r.value['type']}\n")

Expect mem1
Item: mem1; Score (0.5142238669564672)
Memory: I love chocolate ice cream
Type: preference

Expect low score (mem2 not indexed)
Item: mem1; Score (0.4149083716517455)
Memory: I love chocolate ice cream
Type: preference

